In [ ]:
! pip install openpyxl==3.1.2     # installation of "oepnpyxl" library for insertion of data into excel file

# restarting after installation
import os
os._exit(0)

# no any particular verion of Python is required to run this notebook

In [1]:
# making the experiments as much reproducible as possible (in this notebook, all the experiments will be fully reproducible)
seed = 10
from numpy.random import seed as sd
sd(seed)
import random
random.seed(seed)
# import tensorflow as tf
# tf.compat.v1.random.set_random_seed(seed)
import os
os.environ['PYTHONHASHSEED'] = str(seed)
os.environ['CHAINER_SEED'] = str(seed)
# import chainer
# chainer.cuda.cupy.random.seed(seed)
# chainer.backends.cuda.get_device_from_id(gpu).use()
# chainer.backends.cuda.cupy.random.seed(seed)
# pyrandom.seed(seed)
# with chainer.cuda.get_device_from_id(gpu):
#   chainer.cuda.cupy.random.seed(seed)

# importing of required packages and modules
import numpy as np
from math import floor
from math import exp                                                              # Exponential function from the "math"library is used to calculate the exponential in the non-linear equations for calculating the deprivation cost and terminal panelty cost.
import itertools as it
import scipy.stats                                                      # This library is used to generate the demand values using the truncated normal distrubution for a particular episode.
import copy
import time                                                           # Exponential function from the "math"library is used to calculate the exponential in the non-linear equations for calculating the deprivation cost and terminal panelty cost.
import openpyxl

# **Model Parameters**

In [2]:
# inorder to run different problem instances, change these parameters accordingly

accessibality_based_delivery_cost_weight = 1 / 3      # weight used in the weighted sum method for accessibility based delivery cost
deprivation_cost_weight = 1 / 3                       # weight used in the weighted sum method for deprivation cost

no_aa = 4    # number of affected areas
number_of_allocation_time_periods = 12    # number of time periods
cap_dist = np.array([2])          # amount of relief in local resposnse center in one time period

no_materials = 1                  # number of relief items, considered one in all of my experiments

Len = 4       # length (in terms of hours) of one time period

aaa = np.array([2.04 for i in range(0, no_aa * no_materials)])                       # first deprivation parameter used in the deprivation cost function
bbb = np.array([0.24 for i in range(0, no_aa * no_materials)])                       # Second deprivation parameter used in the deprivation cost function

cost_path = np.array([np.array([200 + 50 * i for i in range(0, no_aa * no_materials)]) for i in range(0, number_of_allocation_time_periods)]).reshape(number_of_allocation_time_periods, no_aa * no_materials)      # unit capacity's accessibilities for each affected area and local response center

demand_aa_mat = 1.5 * np.array([np.array([1 for i in range(0, no_aa * no_materials)]) for i in range(0, number_of_allocation_time_periods)]).reshape(number_of_allocation_time_periods, no_aa * no_materials)       # robustified demand generated by an affected area for a relief item in one time period

# **Mounting Google Drive**

In [ ]:
# mount your google drive so that experiment logs can be saved

from google.colab import drive
drive.mount('/content/drive')

# **Markov Decision Process (MDP) Environment for Reinforcement Learning**

In [9]:
no_dcl_loc_aval = 1   # number of local response centers (we are considering just one)
cap_consum_mat = np.array([1]).reshape(no_materials, no_dcl_loc_aval)                   # capacity consumption by a particular relief item in the local response center

is_loc_dec = False          # we are not using location allocation decisions in this research
planning_horizon = Len * number_of_allocation_time_periods

if is_loc_dec:
  erect_cost = np.array([0])                                # fixed cost for the extablishment of local response centers (LRCs), the shape of the array is self.number_of_allocation_time_periods by self.no_dcl_loc_aval. The order is the same.
  removal_cost = np.array([0])                                # fixed cost for the extablishment of local response centers (LRCs), the shape of the array is self.number_of_allocation_time_periods by self.no_dcl_loc_aval. The order is the same.
  # a way to simplify the model and avoid unecessary complications (non-convexities)
  fixed_cost = (erect_cost + removal_cost) / 2        # fixed cost for the extablishment of local response centers (LRCs), the shape of the array is self.number_of_allocation_time_periods by self.no_dcl_loc_aval. The order is the same.

episode_counter = 0

In [10]:
# although the following environment is a generic environment, it is to be used for small-scale instances

class Envir():
  def __init__(self, accessibality_based_delivery_cost_weight = 1 / 2, deprivation_cost_weight = 1 / 2, gamma = 0.99,
               reallocate_t = 5, is_eval = False, is_loc_dec = True, is_use_dyn_prog = False):

        self.is_eval = is_eval
        self.is_loc_dec = is_loc_dec                 # it is a boolean variable which is "True" if location allocation decsions are included otherwise it will be "False"
        self.is_use_dyn_prog = is_use_dyn_prog

        self.accessibality_based_delivery_cost_weight = accessibality_based_delivery_cost_weight

        global no_aa
        global no_materials
        global number_of_allocation_time_periods
        global no_dcl_loc_aval
        global cap_dist
        global cap_consum_mat
        global Len
        global planning_horizon

        if self.is_loc_dec:
          global erect_cost
          global removal_cost
          # a way to simplify the model and avoid unecessary complications (non-convexities)
          global fixed_cost

        global aaa
        global bbb
        global cost_path

        global demand_aa_mat

        if self.is_loc_dec:
          self.fixed_cost_of_dist_centers_weight = accessibality_based_delivery_cost_weight

        self.deprivation_cost_weight = deprivation_cost_weight
        self.terminal_penality_cost_weight = deprivation_cost_weight

        self.gamma = gamma

        if self.is_loc_dec:
          self.reallocate_t = reallocate_t                               # number of time periods after which location realloction decision for local response centers needs to be taken

        self.no_aa = no_aa                                                           # Number of Affected Areas (AAs), to be entered for every instance of the problem.
        self.no_dcl_loc_aval = no_dcl_loc_aval                                                          # number of destrubution centers (DCs) or local response centers, to be entered for every instance of the problem.
        self.no_materials = no_materials
        self.number_of_allocation_time_periods = number_of_allocation_time_periods                                # The number of discrete time periods into which the whole planning time period is to be divided.
        self.steps_per_episode = self.number_of_allocation_time_periods           # Steps per Episodes. (same as above)
        self.cap_dist = cap_dist                            # Capacities of the Distribution Centers or local response center (Simulated by an RL agent). The size of this array is equal to "self.no_materials" times "self.no_dcl".
        self.cap_consum_mat = cap_consum_mat                   # capacity consumption by a particular relief item in a particular local response center
        self.Len = Len       # length (in terms of hours) of one time period
        self.planning_horizon = self.Len * self.number_of_allocation_time_periods

        if self.is_loc_dec:
          self.erect_cost = np.array([0])                                # fixed cost for the extablishment of local response centers (LRCs), the shape of the array is self.number_of_allocation_time_periods by self.no_dcl_loc_aval. The order is the same.
          self.removal_cost = np.array([0])                                # fixed cost for the extablishment of local response centers (LRCs), the shape of the array is self.number_of_allocation_time_periods by self.no_dcl_loc_aval. The order is the same.
          # a way to simplify the model and avoid unecessary complications (non-convexities)
          self.fixed_cost = (self.erect_cost + self.removal_cost) / 2        # fixed cost for the extablishment of local response centers (LRCs), the shape of the array is self.number_of_allocation_time_periods by self.no_dcl_loc_aval. The order is the same.

        self.episode_counter = 0
        self.aaa = aaa                       # The shape is of the form self.no_aa by self.no_materials.                        # In the numpy array the cofficient values for different materials for the same affected area are grouped together. The size of this array is equal to "self.no_materials" times "self.no_dcl_loc_aval".                                       # A constant used in calculating reward for the Agent (Depreivation cost and Terminal Penality cost).
        self.bbb = bbb                       # The shape is of the form self.no_aa by self.no_materials.                        # In the numpy array the cofficient values for different materials for the same affected area are grouped together. The size of this array is equal to "self.no_materials" times "self.no_dcl_loc_aval".                                      # A constant used in calculating reward for the Agent (Depreivation cost and Terminal Penality cost).
        self.cost_path = cost_path

        self.demand_aa_mat = demand_aa_mat                 # In each time period, there is demand associated with each affected area and material. For each time period, the demand is arranged by affected areas and material.

        # all or nothing disruption probabilities for potential LRC location
        if self.is_loc_dec:
          self.disrup_prob = np.array([[0, 1]])         # this must be two dimensional array with size equal to number of local response centers time "2" ("2" because there can be only two possible scenarios for an LRC, either it is disrupted or it is not disrupted)
          self.loc_disrup_temp = np.zeros((self.no_dcl_loc_aval,))          # this array will be used in disruption calculations

        # enumeration of location decisions for all local response centers or distribution centers
        if self.is_loc_dec:
          self.loc_dcl = []
          a = 0
          while a < self.no_dcl_loc_aval:
            self.loc_dcl.append([0, 1])
            a = a + 1
          self.loc_dcl = np.array(self.loc_dcl)
          self.loc_dcl_actions = np.array(np.meshgrid(*list(self.loc_dcl))).T.reshape(-1, self.no_dcl_loc_aval)      # len([0, 1]) = 2                     # All the possible actions related to loction are listed in "self.loc_dcl_actions".
          del self.loc_dcl                                     # Deleting the 'self.loc_dcl' array as it is no longer required, to free up memory. (Make the code memory efficient)
          del a

          self.dc_loc_alr_const = np.array([1])                             # The length of the array is equal to the number of distrubution centers, "self.no_dcl_loc_aval".                   # The array representing the agent's decision for establishing distribution center at a particular avalilable location (to be taken from agent's action array).

        # enumeration of resource allocation decision from relief material allocation from local response centers to affected areas
        self.counter = 0                                                                  # Its value will increase until all the combinations of the affected areas and distrubuiton centers are exhausted.

        self.res_alocat_dc = []
        m = 0
        while m < self.no_aa:                                             # These nested while loops have the task to generate the numpy array list of all possible allocation aciton, which the agent can do. In it, there may be some actions which are infeasible.
          i = 0
          while i < self.no_materials:
            j = 0
            while j < self.no_dcl_loc_aval:
              k = 0
              self.res_alocat_dc.append([])
              while k <= np.floor(self.cap_dist[j] / self.cap_consum_mat[i, j]):
                self.res_alocat_dc[self.counter].append(k)
                k = k + 1
              self.counter = self.counter + 1
              j = j + 1
            i = i + 1
          m = m + 1
        self.res_alocat_dc = np.array(self.res_alocat_dc)
        self.alot_dcl_actions = np.array(np.meshgrid(*self.res_alocat_dc)).T.reshape(-1, self.no_dcl_loc_aval * self.no_aa * self.no_materials)                           # This statement creates the real numpy array list of all the possible resource allocation decisions. "self.alot_dcl_actions" contains all the possible actions related to resource allocation. The arrangement in "self.alot_dcl_actions" is self.no_materials by self.no_dcl_loc_aval.

        # determination of action validaty (are all constraints satisfied or not?) and the elimination of invalid actions
        self.del_el = []                  # array elements to be deleated
        # if the allocation capacity of any distribution center location is exceeded by the total allocation made from that distribution ceneter location then the action is eliminated in the following code.
        for j in range(0, self.alot_dcl_actions.shape[0]):          # Action shaping implementation. (eliminating some obvious infeasible actions) Ref: A. Kanervisto, C. Scheller and V. Hautamäki, "Action Space Shaping in Deep Reinforcement Learning," 2020 IEEE Conference on Games (CoG), 2020, pp. 479-486, doi: 10.1109/CoG47356.2020.9231687.
          self.compare_for_action_shaping = np.sum(np.multiply(np.reshape(np.sum(np.reshape(self.alot_dcl_actions[j], (self.no_aa, self.no_materials * self.no_dcl_loc_aval)), axis = 0), (self.no_materials, self.no_dcl_loc_aval)), self.cap_consum_mat), axis = 0)               # As capacity constraint is the hard constraint so we use action shaping to eliminate the actions which violate this constraint to ensure that the capacity constraints must be satisfied, otherwise it makes no sense that the allocation exceeds the capacity of distribution centers.
          for k in range(0, self.compare_for_action_shaping.shape[0]):
            if self.compare_for_action_shaping[k] > self.cap_dist[k]:
              self.del_el.append(j)
              break

        self.alot_dcl_actions = np.delete(self.alot_dcl_actions, self.del_el, axis = 0)

        del self.counter
        del m
        del i
        del j
        del k
        del self.res_alocat_dc                                                          # Deleting the variables array as it is no longer required, to free up memory. (Make the code memory efficient)

        if self.is_loc_dec:
          i = 0
          while i < 2 ** self.no_dcl_loc_aval:                                          # Number of possible location allocation decisions are equal to 2 raise to power the total number of distrubution centers.
            self.temp_arr_loc = np.repeat(self.loc_dcl_actions[i].reshape(1, -1), self.alot_dcl_actions.shape[0], axis = 0)
            self.alot_loc_dcl_actions_full = np.concatenate((self.temp_arr_loc, np.copy(self.alot_dcl_actions)), axis = 1)
            if i == 0:
              self.alot_loc_dcl_actions_full_array = self.alot_loc_dcl_actions_full
            else:
              self.alot_loc_dcl_actions_full_array = np.concatenate((self.alot_loc_dcl_actions_full_array, self.alot_loc_dcl_actions_full), axis = 0)                              # "self.alot_loc_dcl_actions_full_array" numpy array contains all the possible action (location and resoruce allocation actions combined)
            i = i + 1

          del self.alot_loc_dcl_actions_full

          # filteration based on location allocation and reallocation deicsions
          self.del_el_loc_constr = []                  # array elements to be deleated
          i = 0
          # if any allocation is made through the distribution center location such that this distribution center location is not used in a particular action then these actions are eliminated (deleted).
          while i < self.alot_loc_dcl_actions_full_array.shape[0]:                                              # Action shaping implementation. (eliminating some obvious infeasible actions) Ref: A. Kanervisto, C. Scheller and V. Hautamäki, "Action Space Shaping in Deep Reinforcement Learning," 2020 IEEE Conference on Games (CoG), 2020, pp. 479-486, doi: 10.1109/CoG47356.2020.9231687.
            self.alot_loc_dcl_actions_full_array_loc = self.alot_loc_dcl_actions_full_array[i, 0 : self.no_dcl_loc_aval]
            self.alot_loc_dcl_actions_full_array_res = self.alot_loc_dcl_actions_full_array[i, self.no_dcl_loc_aval:].reshape(-1, self.no_dcl_loc_aval)
            self.alot_loc_dcl_actions_full_array_res = np.sum(self.alot_loc_dcl_actions_full_array_res, axis = 0)
            for j in range(0, self.no_dcl_loc_aval):
              if self.alot_loc_dcl_actions_full_array_loc[j] == 0 and self.alot_loc_dcl_actions_full_array_res[j] != 0:
                self.del_el_loc_constr.append(i)
                break
            i = i + 1

          self.alot_loc_dcl_actions_full_array = np.delete(self.alot_loc_dcl_actions_full_array, self.del_el_loc_constr, axis = 0)
        else:
         self.alot_loc_dcl_actions_full_array = self.alot_dcl_actions   # if location allocation and relocation decisions are not included, then all the possible actions are only valid resource allocation actions
        # enumeration of environment's state

        # the following is an array which will store all the possible states in the environment
        if self.is_loc_dec:
          # upper bound on the number of possible state
          self.upper_bound_state_no = np.prod(self.number_of_allocation_time_periods * np.zeros_like(self.demand_aa_mat[0]) + 1) * (2 ** self.no_dcl_loc_aval) * self.number_of_allocation_time_periods
          self.state_storg_arr = np.zeros((self.upper_bound_state_no , self.no_aa * self.no_materials + self.no_dcl_loc_aval + 1))
        else:
          # upper bound on the number of possible state
          self.upper_bound_state_no = np.prod(self.number_of_allocation_time_periods * np.zeros_like(self.demand_aa_mat[0]) + 1 + np.abs(self.number_of_allocation_time_periods * np.zeros_like(self.demand_aa_mat[0]) - np.floor((np.max(self.cap_dist) / np.min(self.cap_consum_mat))) * self.number_of_allocation_time_periods)) * self.number_of_allocation_time_periods
          self.state_storg_arr = np.zeros((int(self.upper_bound_state_no) , self.no_aa * self.no_materials + 1))

        self.state_lst_storg = [[] for i in range(0, self.number_of_allocation_time_periods + 1)]     # this list of list will store only in the indices of the state arrays in "self.state_storg_arr" with respect to each time period
        self.init_state = np.array([0 for i in range(0, self.no_aa * self.no_materials)])

        if self.is_use_dyn_prog:
          self.prob_vec = np.zeros((self.upper_bound_state_no, self.alot_loc_dcl_actions_full_array.shape[0], self.upper_bound_state_no))

        counter_1 = 0       # this counter tracks the index in "self.state_storg_arr" where the states from the previous time period (which will be used along with actions to enumerate states for the current time period in our finite horizon MDP) starts
        counter_2 = 0       # this counter tracks the index in "self.state_storg_arr"  where state from the previous the previous time period ends
        counter_3 = 1       # this counter starts at the index where the next state (of the current time period) is to entered in "self.state_storg_arr"
        for i in range(0, self.number_of_allocation_time_periods):           # loop over all the time periods
          for j in range(counter_1, counter_2 + 1):      # loop over all the states of the previous time period (i.e., between the index "counter_1" and "counter_2")
            for k in range(0, self.alot_loc_dcl_actions_full_array.shape[0]):       # loop over all the actions which the agent can take
              if self.is_loc_dec:
                # computation of location allocation decision
                if i % self.reallocate_t == 0:
                  # the following code will be executed if the feedback policy is being evaluated
                  self.loc_state = self.alot_loc_dcl_actions_full_array[k, :self.no_dcl_loc_aval]
                else:
                  self.loc_state = self.state_storg_arr[j, self.no_aa * self.no_materials : self.no_aa * self.no_materials + self.no_dcl_loc_aval]

                # if disruption is not considered the state enumeration is much simpler
                if np.sum(self.disrup_prob[:, 0].reshape(-1,), axis = 0) != 0:
                  # modification of local response centers' sub state to take into account disruption risk
                  if np.sum(self.loc_state) != 0:
                    next_state_store = np.array(np.meshgrid(*[[0, 1] for i in range(0, int(np.sum(self.loc_state)))])).T.reshape(-1, int(np.sum(self.loc_state)))
                    indx_non_zero_next_st = np.nonzero(self.loc_state)[0]
                    for indx_l, l in enumerate(next_state_store):
                      prob_temp = 1
                      loc_sub_state_st = copy.deepcopy(self.loc_state)
                      for m in range(0, len(indx_non_zero_next_st)):
                        # print(self.loc_state, "lllllllll")
                        # print(indx_non_zero_next_st, "yyyyyyyyyyyy")
                        loc_sub_state_st[int(indx_non_zero_next_st[m])] = l[m]
                        prob_temp = prob_temp * self.disrup_prob[int(indx_non_zero_next_st[m]), l[m]]
                        # print(self.demand_aa_mat[i], "333333333333")
                        state_temp_temp = np.concatenate((self.state_storg_arr[j, 0 : self.no_aa * self.no_materials] - np.sum(np.reshape(self.alot_loc_dcl_actions_full_array[k, self.no_dcl_loc_aval:], (self.no_aa * self.no_materials, self.no_dcl_loc_aval)), axis = 1) + self.demand_aa_mat[i], loc_sub_state_st, np.array([i + 1])), axis = 0)
                        # print(state_temp_temp, "state_temp_tempstate_temp_tempstate_temp_temp")
                        # print(self.state_storg_arr[counter_2 : counter_3,:], "3333333333333333333")
                        if np.sum(np.prod(self.state_storg_arr[counter_2 : counter_3,:] == state_temp_temp, axis = 1)) == 0:
                          self.state_storg_arr[counter_3] = state_temp_temp
                          self.state_lst_storg[i].append(counter_3)
                          if self.is_use_dyn_prog:
                            self.prob_vec[j, k, counter_3] = prob_temp     # forming of the state trasition probability matrix (3D array with size as number of states times number of actions times number of states)
                            print(self.prob_vec[j, k, counter_3], "ppppppppppppp")
                            print(j, k, counter_3)
                          # print(prob_temp, "prob_tempprob_tempprob_temp")
                          counter_3 = counter_3 + 1
                  else:
                    state_temp_temp = np.concatenate((self.state_storg_arr[j, 0 : self.no_aa * self.no_materials] - np.sum(np.reshape(self.alot_loc_dcl_actions_full_array[k, self.no_dcl_loc_aval:], (self.no_aa * self.no_materials, self.no_dcl_loc_aval)), axis = 1) + self.demand_aa_mat[i], self.loc_state, np.array([i + 1])), axis = 0)
                    # print(state_temp_temp, "state_temp_tempstate_temp_tempstate_temp_temp")
                    # print(self.state_storg_arr[counter_2 : counter_3,:], "3333333333333333333")
                    if np.sum(np.prod(self.state_storg_arr[counter_2 : counter_3,:] == state_temp_temp, axis = 1)) == 0:
                      self.state_storg_arr[counter_3] = state_temp_temp
                      self.state_lst_storg[i].append(counter_3)
                      if self.is_use_dyn_prog:
                        self.prob_vec[j, k, counter_3] = 1

                      counter_3 = counter_3 + 1
                else:
                  state_temp_temp = np.concatenate((self.state_storg_arr[j, 0 : self.no_aa * self.no_materials] - np.sum(np.reshape(self.alot_loc_dcl_actions_full_array[k, self.no_dcl_loc_aval:], (self.no_aa * self.no_materials, self.no_dcl_loc_aval)), axis = 1) + self.demand_aa_mat[i], self.loc_state, np.array([i + 1])), axis = 0)
                  # print(state_temp_temp, "state_temp_tempstate_temp_tempstate_temp_temp")
                  # print(self.state_storg_arr[counter_2 : counter_3,:], "3333333333333333333")
                  if np.sum(np.prod(self.state_storg_arr[counter_2 : counter_3,:] == state_temp_temp, axis = 1)) == 0:
                    self.state_storg_arr[counter_3] = state_temp_temp
                    self.state_lst_storg[i].append(counter_3)
                    if self.is_use_dyn_prog:
                      self.prob_vec[j, k, counter_3] = 1

                    counter_3 = counter_3 + 1

              else:
                state_temp_temp = np.concatenate((self.state_storg_arr[j, 0 : self.no_aa * self.no_materials] - np.sum(np.reshape(self.alot_loc_dcl_actions_full_array[k], (self.no_aa * self.no_materials, self.no_dcl_loc_aval)), axis = 1) + self.demand_aa_mat[i], np.array([i + 1])), axis = 0)
                if np.sum(np.prod(self.state_storg_arr[counter_2 : counter_3,:] == state_temp_temp, axis = 1)) == 0:
                  self.state_storg_arr[counter_3] = state_temp_temp
                  self.state_lst_storg[i].append(counter_3)
                  if self.is_use_dyn_prog:
                    self.prob_vec[j, k, counter_3] = 1

                  counter_3 = counter_3 + 1
                  # self.state_storg_arr[: counter_3 - 1, :] = np.unique(self.state_storg_arr[: counter_3 - 1, :], axis = 0)
                  # print(counter_3, "counter_2counter_2counter_2")
          counter_1 = counter_2 + 1
          counter_2 = counter_3 - 1

        self.counter_1 = counter_1
        self.state_storg_arr = self.state_storg_arr[: counter_3]      # removing all the rows in state storage array which are not used after all the possible states have been enumerated
        # self.state_storg_arr = np.unique(self.state_storg_arr, axis = 0)    # removing all the rows which are not unique (untile they are unique)

        self.Ns = self.state_storg_arr.shape[0]
        self.Na = self.alot_loc_dcl_actions_full_array.shape[0]

        if self.is_use_dyn_prog:
          self.prob_vec = self.prob_vec[:self.counter_1, :, :self.Ns]

  def step(self, act):                                                            # The agent just gives the index of the action which is taken by the RL agent, as the actions are discrete.

        self.sstep = self.sstep + 1                                                   # The above statement is there as a counter. It countes the number of steps taken in an episode. (updating of the step counter.)

        self.act_raw = self.alot_loc_dcl_actions_full_array[act]              # Taking the action vector from the agent and storing it in the "step" class attribute "act".                                                    # Step counter, which counts the number of steps in an episodes.

        if self.is_loc_dec:
          self.act_loc = self.act_raw[:self.no_dcl_loc_aval]              # location allocation decision for local response centers (LRCs)

          self.act_partial_raw = self.act_raw[self.no_dcl_loc_aval:]           # This statement extracts only the resoruce allocation portion of the action taken by the agent.

        else:
          self.act_partial_raw = self.act_raw

        # the following conditional state will be executed only when the agent cannot make decisions on locations of LRCs. Thus, we must make the resoruce allocation actions feasible with the already made location allocation decisons in the previous time period (modified by disruption)
        # making resource allocation action feasible to what LRCs locations are currently operational (for location allocation decisions less time periods)
        if self.is_loc_dec and ((self.sstep - 1) % self.reallocate_t != 0):
          # modification of "self.act_partial_raw"
          self.act_partial_raw = np.multiply(np.tile(self.loc_state_temp, self.no_aa * self.no_materials).reshape(self.no_aa * self.no_materials, self.no_dcl_loc_aval), np.reshape(self.act_raw[self.no_dcl_loc_aval:], (self.no_aa * self.no_materials, self.no_dcl_loc_aval))).reshape(-1,)

        # the following code will be executed when the trained feedback policy is being evaluated
        if self.is_eval:
          print("resource_allocation_decisions: ", self.act_partial_raw)

        self.act_partial_raw_reshape = np.reshape(self.act_partial_raw, (self.no_aa, self.no_materials * self.no_dcl_loc_aval))
        self.act_partial_raw_reshape_sum = np.sum(self.act_partial_raw_reshape, axis = 0)

        # transition function
        self.state = self.state - np.sum(np.reshape(self.act_partial_raw, (self.no_aa * self.no_materials, self.no_dcl_loc_aval)), axis = 1) + self.demand_aa_mat[self.sstep - 1]                  # The computation of the next state vector.

        # local response centers' location update (part of transition function)
        if self.is_loc_dec:
          if (self.sstep - 1) % self.reallocate_t == 0:

            # the following code will be executed if the the feedback policy is being evaluated
            if self.is_eval:
              print("LRCs'_location_allocation_and_relocation_decisions: ", self.act_loc)

            self.loc_state = self.act_loc
            print(self.act_loc, "llllllllllllll")
          else:
            self.loc_state = self.loc_state_temp

          # self.loc_state = np.array([1 for i in range(0, self.no_dcl_loc_aval)])       # this statement will run only when location allocation decisons are not to be included

        i = 0
        self.act = []
        self.accumulate = 0

        self.cap_dist_matrix = self.cap_dist.reshape(-1, self.no_dcl_loc_aval)

        # fixied cost for establishment of local response centers (LRCs) (installation or deinstallation cost)
        if self.is_loc_dec:
          self.fixed_cost_of_dist_centers = np.dot(np.abs(self.loc_state - self.loc_state_temp), self.fixed_cost)
          if self.sstep == self.steps_per_episode:              # at the end of planning horizon, all the established location response centers (LRC) (as they are the temprary strcutures)
            self.fixed_cost_of_dist_centers = self.fixed_cost_of_dist_centers + np.dot(self.loc_state, self.fixed_cost)            # at the of planning horizon (episode) the removal cost for all established local response centers (LRCs) must be included in the incurred fixed cost

          self.loc_state_temp = copy.deepcopy(self.loc_state)

        # before the system can move two the next time step (and after the complete execution of current time period's decisions) LRCs are exposed to all or nothing disruption
        if self.is_loc_dec:
          for i in range(0, self.no_dcl_loc_aval):
            self.loc_disrup_temp[i] = np.random.choice(np.array([0, 1]), size=(1,), replace=True, p=self.disrup_prob[i])
          self.loc_state_temp = np.multiply(self.loc_state_temp, self.loc_disrup_temp).reshape(-1,)

          # the following code will be executed if the the feedback policy is being evaluated
          if self.is_eval:
            print("which_potential_LRCs'_locations_have_disruption_occured_on_them_in_the_current_time_period_which_will_impact_subsequent_time_period?: ", self.loc_disrup_temp)

        counter = 0

        self.accessibility_based_delivery_cost = np.dot(self.act_partial_raw, self.cost_path[self.sstep - 1])               # "Accesibility Based Delivery Cost" is calculated by computing the dot product of allocation action vector, and the path cost vector.

        self.dep_cost_sum = 0                                            # Before calculating the total deprivation cost, the class attribute "deprivation cost" is set to "0".
        if self.sstep != self.steps_per_episode:
          for s in self.state:                                               # This for loop loops through all the affected areas and calculate the total deprivation cost.
            if s >= 0:                                                                   # It the state value of the affected area is greater then or equal to "0" then the formula below is used.
              self.dep = exp(self.aaa[counter]) * (exp(self.bbb[counter] * self.Len) - 1) * (exp(self.bbb[counter] * self.Len) ** s)                        # This formula is used to calculate the deprivation cost for a particular Affected Area, given a particular state value of that Affected Area.
            else:
              self.dep = 0                                                                 # If the state value of the Affected Area is less then "0" then the deprivation cost of that affected area will be set to "0".
            if self.sstep == 1:                                                              # If it is the first time step of an episode then there is an additional step is calculating the deprivation cost for the time period.
              self.dep = self.dep + np.multiply(np.exp(self.aaa[counter]), (np.exp(self.bbb[counter] * self.Len) - 1))               # This is the additional step is calculating he deprivation cost for the first time period of the episode.

            counter = counter + 1

            self.dep_cost_sum = self.dep_cost_sum + self.dep


        else:                                     # If the time step of and episode is the last time step then there is an additional cost called the "penelty cost".
          self.terminal_panelty_cost_sum = 0                                                # Before calculating the terminal panelty cost, the class attribute "terminal_penelty_cost" is set to zero.
          counter = 0
          for s in self.state:                                                         # This for loop loop through all the Affected Areas to calculate the terminal penelty cost of each Affected Area and total terminal penelty cost.
            if s >= 0:                                                                   # If the state value of the Affected Area is greater then or equal to "0" then the formula is the next statement is used to calcualte the terminal penelty cost.
              self.term = exp(self.aaa[counter]) * (exp(self.bbb[counter] * self.Len)-1) * (exp(self.bbb[counter] * self.Len) ** s)              # The formula used for calculating the terminal penelty cost for a particular Affected Area.
            else:
              self.term = 0                                                              # If state value of the affected area is less then "0" then the terminal panelty cost is set to zero.

            counter = counter + 1

            self.terminal_panelty_cost_sum = self.terminal_panelty_cost_sum + self.term

        # The following if-elif-else computes the reward which will be delivered to the agent.
        if self.is_loc_dec:
          if self.sstep != self.steps_per_episode:
            self.reward = - self.deprivation_cost_weight * self.dep_cost_sum - self.accessibality_based_delivery_cost_weight * self.accessibility_based_delivery_cost - self.accessibality_based_delivery_cost_weight * self.fixed_cost_of_dist_centers                 # If the Markovian time step is the first time step, then the reward is calculated by summing the negative of total deprivations cost and total accessibility based delivery cost.
          elif self.sstep == self.steps_per_episode:
            self.reward = - self.deprivation_cost_weight * self.terminal_panelty_cost_sum - self.accessibality_based_delivery_cost_weight * self.accessibility_based_delivery_cost - self.accessibality_based_delivery_cost_weight * self.fixed_cost_of_dist_centers             # If the Markovian time step is the last time step then the reward for the RL agent is calculated by summing the negative of accessibility based delivery cost and terminal penality cost.

          del self.fixed_cost_of_dist_centers

        else:
          if self.sstep != self.steps_per_episode:
            self.reward = - self.deprivation_cost_weight * self.dep_cost_sum - self.accessibality_based_delivery_cost_weight * self.accessibility_based_delivery_cost                 # If the Markovian time step is the first time step, then the reward is calculated by summing the negative of total deprivations cost and total accessibility based delivery cost.
          elif self.sstep == self.steps_per_episode:
            self.reward = - self.deprivation_cost_weight * self.terminal_panelty_cost_sum - self.accessibality_based_delivery_cost_weight * self.accessibility_based_delivery_cost             # If the Markovian time step is the last time step then the reward for the RL agent is calculated by summing the negative of accessibility based delivery cost and terminal penality cost.

        del self.accessibility_based_delivery_cost

        if self.sstep == self.steps_per_episode:
          self.done = True                                                                 # If the time period is the last time period of the episode then set the class attribute "done" to True.
        else:
          self.done = False                                                                # If the time period is not the last time period of the episode then set the class attribute "done" to False.
        self.info = {}                                                                     # Class attribute "info" is an empty python dictionary which is returned to the RL agent at every time step.

        # the following code will be executed if the the feedback policy is being evaluated
        if self.is_eval:
          print("reward_or_stagewise_objective_function_value: ", self.reward)

        if self.is_loc_dec:
          self.state_indx_curr = np.where(np.all(self.state_storg_arr == np.concatenate((self.state.reshape(self.no_aa * self.no_materials,), self.loc_state_temp, np.array([self.sstep])), axis = 0), axis = 1))[0][0]
          return self.state_indx_curr, float(self.reward), self.done, self.info           # The step method will return the state of the time period, the reward, "done" attribute (which shows weather the episode has terminated or not) and the empty python dictionary "info".
        else:
          self.state_indx_curr = np.where(np.all(self.state_storg_arr == np.concatenate((self.state.reshape(self.no_aa * self.no_materials,), np.array([self.sstep])), axis = 0), axis = 1))[0][0]
          return self.state_indx_curr, float(self.reward), self.done, self.info           # The step method will return the state of the time period, the reward, "done" attribute (which shows weather the episode has terminated or not) and the empty python dictionary "info".


  def render(self):                                                               # The "render" method is used to render the environment or to visually simulate the RL environment. It is computationally costly to render the environment and also it is not necessary to render this environment. This method is mostly used for debug the environment.
        pass                                                                      # Here, "pass" keyword is used to just pass the method, this method () is not being used (Environment is not rendered).

  def reset(self):                                                                # The "reset" method, which is called when a new episode is started. It gives the initial state to the RL agent.

        self.sstep = 0                                                            # The step counter initialized at zero.
        self.state = np.array([0 for i in range(0, self.no_aa * self.no_materials)])                                               # Initial state, which is zero for every affected area and material, Arrangement is affected areas and then number of materials.

        if self.is_loc_dec:
          self.loc_state_temp = np.array([0 for i in range(0, self.no_dcl_loc_aval)])        # assuming no local response centers are pre-eracted
          self.state_indx_curr = np.where(np.all(self.state_storg_arr == np.concatenate((self.state, self.loc_state_temp, np.array([self.sstep])), axis = 0), axis=1))[0][0]
          return self.state_indx_curr
        else:
          self.state_indx_curr = np.where(np.all(self.state_storg_arr == np.concatenate((self.state, np.array([self.sstep])), axis = 0), axis=1))[0][0]
          return self.state_indx_curr

# **Dynamic Programming (exploiting the fact that our MDP is of finite horizon)**

In [8]:
# creating te excel sheet for insertion of data using "openpyxl"

wb = openpyxl.Workbook()     # creating a new workbook
ws = wb.active   # activating the workbook for reading and writing of data into and from the workbook respectively
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))

In [ ]:
is_loc_dec = False         # we are not using location allocation decisions in this research

gamma = 0.99999           # discount factor

no_dcl_loc_aval = 1        # number of local response centers (we are considering just one)
cap_consum_mat = np.array([1]).reshape(no_materials, no_dcl_loc_aval)                   # capacity consumption by a particular relief item in a particular local response center
Len = 4       # length (in terms of hours) of one time period
planning_horizon = Len * number_of_allocation_time_periods

episode_counter = 0


# enumeration of resource allocation decision from relief material allocation from local response centers to affected areas
counter = 0                                                                  # Its value will increase until all the combinations of the affected areas and distrubuiton centers are exhausted.

res_alocat_dc = []
m = 0
while m < no_aa:                                             # These nested while loops have the task to generate the numpy array list of all possible allocation aciton, which the agent can do. In it, there may be some actions which are infeasible.
  i = 0
  while i < no_materials:
    j = 0
    while j < no_dcl_loc_aval:
      k = 0
      res_alocat_dc.append([])
      while k <= np.floor(cap_dist[j] / cap_consum_mat[i, j]):
        res_alocat_dc[counter].append(k)
        k = k + 1
      counter = counter + 1
      j = j + 1
    i = i + 1
  m = m + 1
res_alocat_dc = np.array(res_alocat_dc)
alot_dcl_actions = np.array(np.meshgrid(*res_alocat_dc)).T.reshape(-1, no_dcl_loc_aval * no_aa * no_materials)                           # This statement creates the real numpy array list of all the possible resource allocation decisions. "alot_dcl_actions" contains all the possible actions related to resource allocation. The arrangement in "alot_dcl_actions" is no_materials by no_dcl_loc_aval.

# determination of action validaty (are all constraints satisfied or not?) and the elimination of invalid actions
del_el = []                  # array elements to be deleated
# if the allocation capacity of any distribution center location is exceeded by the total allocation made from that distribution ceneter location then the action is eliminated in the following code.
for j in range(0, alot_dcl_actions.shape[0]):          # Action shaping implementation. (eliminating some obvious infeasible actions) Ref: A. Kanervisto, C. Scheller and V. Hautamäki, "Action Space Shaping in Deep Reinforcement Learning," 2020 IEEE Conference on Games (CoG), 2020, pp. 479-486, doi: 10.1109/CoG47356.2020.9231687.
  compare_for_action_shaping = np.sum(np.multiply(np.reshape(np.sum(np.reshape(alot_dcl_actions[j], (no_aa, no_materials * no_dcl_loc_aval)), axis = 0), (no_materials, no_dcl_loc_aval)), cap_consum_mat), axis = 0)               # As capacity constraint is the hard constraint so we use action shaping to eliminate the actions which violate this constraint to ensure that the capacity constraints must be satisfied, otherwise it makes no sense that the allocation exceeds the capacity of distribution centers.
  for k in range(0, compare_for_action_shaping.shape[0]):
    if compare_for_action_shaping[k] > cap_dist[k]:
      del_el.append(j)
      break

alot_dcl_actions = np.delete(alot_dcl_actions, del_el, axis = 0)

del counter
del m
del i
del j
del k
del res_alocat_dc                                                          # Deleting the variables array as it is no longer required, to free up memory. (Make the code memory efficient)

alot_loc_dcl_actions_full_array = alot_dcl_actions   # if location allocation and relocation decisions are not included, then all the possible actions are only valid resource allocation actions

# enumeration of environment's state

# upper bound on the number of possible state
upper_bound_state_no = np.prod(number_of_allocation_time_periods * np.zeros_like(demand_aa_mat[0]) + 1 + np.abs(number_of_allocation_time_periods * np.zeros_like(demand_aa_mat[0]) - np.floor((np.max(cap_dist) / np.min(cap_consum_mat))) * number_of_allocation_time_periods)) * number_of_allocation_time_periods
state_storg_arr = np.zeros((int(upper_bound_state_no) , no_aa * no_materials + 1))

state_lst_storg = [[] for i in range(0, number_of_allocation_time_periods + 1)]     # this list of list will store only in the indices of the state arrays in "state_storg_arr" with respect to each time period
init_state = np.array([0 for i in range(0, no_aa * no_materials)])

counter_1 = 0       # this counter tracks the index in "state_storg_arr" where the states from the previous time period (which will be used along with actions to enumerate states for the current time period in our finite horizon MDP) starts
counter_2 = 0       # this counter tracks the index in "state_storg_arr"  where state from the previous the previous time period ends
counter_3 = 1       # this counter starts at the index where the next state (of the current time period) is to entered in "state_storg_arr"
for i in range(0, number_of_allocation_time_periods):           # loop over all the time periods
  for j in range(counter_1, counter_2 + 1):      # loop over all the states of the previous time period (i.e., between the index "counter_1" and "counter_2")
    for k in range(0, alot_loc_dcl_actions_full_array.shape[0]):       # loop over all the actions which the agent can take
      state_temp_temp = np.concatenate((state_storg_arr[j, 0 : no_aa * no_materials] - np.sum(np.reshape(alot_loc_dcl_actions_full_array[k], (no_aa * no_materials, no_dcl_loc_aval)), axis = 1) + demand_aa_mat[i], np.array([i + 1])), axis = 0)
      if np.sum(np.prod(state_storg_arr[counter_2 : counter_3,:] == state_temp_temp, axis = 1)) == 0:
        state_storg_arr[counter_3] = state_temp_temp
        state_lst_storg[i].append(counter_3)

        counter_3 = counter_3 + 1
        # state_storg_arr[: counter_3 - 1, :] = np.unique(state_storg_arr[: counter_3 - 1, :], axis = 0)
        # print(counter_3, "counter_2counter_2counter_2")
  counter_1 = counter_2 + 1
  counter_2 = counter_3 - 1

counter_1 = counter_1
state_storg_arr = state_storg_arr[: counter_3]      # removing all the rows in state storage array which are not used after all the possible states have been enumerated
# state_storg_arr = np.unique(state_storg_arr, axis = 0)    # removing all the rows which are not unique (untile they are unique)

state_lst_storg.insert(0, [0])

Ns = state_storg_arr.shape[0]
Na = alot_loc_dcl_actions_full_array.shape[0]

planning_horizon = Len * number_of_allocation_time_periods

In [ ]:
# a separate reward function for the environment
def reward_func(s, a, s_):
  # accessibility based delivery cost
  logistics_cost = np.dot(cost_path[int(state_storg_arr[s, -1]) - 1], alot_loc_dcl_actions_full_array[a])

  # deprivation cost
  deprivation_cost = 0
  counter = 0
  for st in state_storg_arr[s_, : no_aa * no_materials]:           # This for loop loops through all the affected areas and calculate the total deprivation cost.
    if st >= 0:                                                                   # It the state value of the affected area is greater then or equal to "0" then the formula below is used.
      deprivation_cost = deprivation_cost + exp(aaa[counter]) * (exp(bbb[counter] * Len) - 1) * (exp(bbb[counter] * Len) ** st)                        # This formula is used to calculate the deprivation cost for a particular Affected Area, given a particular state value of that Affected Area.
    if state_storg_arr[s, -1] == 0:                                                              # If it is the first time step of an episode then there is an additional step is calculating the deprivation cost for the time period.
      deprivation_cost = deprivation_cost + np.multiply(np.exp(aaa[counter]), (np.exp(bbb[counter] * Len) - 1))               # This is the additional step is calculating he deprivation cost for the first time period of the episode.
    counter = counter + 1
  return -((1 / 3) * deprivation_cost + (1 / 3) * logistics_cost)

In [ ]:
import numpy as np
import itertools as it

st_time = time.time()

# Calcuation of the value functions corresponding to each state, as well as storing action corresponding to the highest state value for each state at every time step.
state_act_value_function = [np.zeros((len(state_lst_storg[i]), Na)) for i in range(0, number_of_allocation_time_periods + 1)]
for i in range(number_of_allocation_time_periods - 1, -1, -1):     # looping over the episodes but from the end of the episodes to the start of the episode as for calculating state values in dynamic programming the state values are memoized to calculate the state values of the prvious time steps's state
  # calculating the value function for all the states in all the steps in an episode
  # storing the state value along with the optimal action from a particular state
  for indx_j, j in enumerate(state_lst_storg[i]):               # loop over all the states in the current time period
    for k in range(0, alot_loc_dcl_actions_full_array.shape[0]):    # loop over all the actions which can be taken in the current time period
      value = 0
      for indx_l, l in enumerate(state_lst_storg[i + 1]):
        value = value + gamma * np.all(state_storg_arr[l, 0 : no_aa * no_materials] == (state_storg_arr[j, 0 : no_aa * no_materials] - alot_loc_dcl_actions_full_array[k] + demand_aa_mat[i])) * (reward_func(j, k, l) + np.max(state_act_value_function[i + 1][indx_l]))
      state_act_value_function[i][indx_j, k] = value
if i == 0:
  print(value)

for i in range(0, number_of_allocation_time_periods):
  state_act_value_function[i] = np.argmax(state_act_value_function[i], axis = 1)
temp_list = list(state_act_value_function[i] for i in range(0, number_of_allocation_time_periods))
greedy_policy_dyn_prog = np.concatenate(temp_list, axis = 0)
del state_act_value_function

exce_time = time.time() - st_time

In [ ]:
# number of time steps for which te algorithm is trained
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))
ws.append(("Execution time: ", exce_time))
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))

## **Evaluation of Dynamic Programming Solution Policy**

In [ ]:
env = Envir(accessibality_based_delivery_cost_weight = accessibality_based_delivery_cost_weight, deprivation_cost_weight = deprivation_cost_weight, gamma = gamma,
            reallocate_t = 5, is_eval = False, is_loc_dec = False, is_use_dyn_prog = False)

In [ ]:
n_eval_ep = 1

obj_val_dyn_prog = 0

# for storing action during evaluation simulation
act_store = np.zeros((env.number_of_allocation_time_periods, env.no_aa * env.no_materials * env.no_dcl_loc_aval))

# for storing states during evaluation simulation
state_store = np.zeros((env.number_of_allocation_time_periods + 1, env.no_aa * env.no_materials))

ws.append(("Stagewise objective function value or reward:",))

for i in range(0, n_eval_ep):     # loop over the number of episodes
  s = env.reset()     # initial state
  state_store[0, :] = env.state_storg_arr[s, :env.no_aa * env.no_materials]
  for j in range(0, env.number_of_allocation_time_periods):     # loop over all the time peirods inside an episode
    a = greedy_policy_dyn_prog[s]
    act_store[j, :] = env.alot_loc_dcl_actions_full_array[a]
    s, reward, done, info = env.step(a)

    # storing all the stage wise objective values into an excel sheet
    ws.append((f"Stagewise objective function value at stage {j + 1}: ", reward))

    obj_val_dyn_prog = obj_val_dyn_prog + (gamma ** i) * reward
    state_store[j + 1, :] = env.state_storg_arr[s, :env.no_aa * env.no_materials]

In [ ]:
act_store

In [ ]:
# storing all the objective function valeu (sum of all the rewards) into an excel sheet
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))
ws.append(("Objective function value: ", obj_val_dyn_prog))
obj_val_dyn_prog
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))

In [ ]:
# storing all the state variables value to an excel sheet
ws.append(("State Variables",))
for i in state_store:
  ws.append(tuple(i))

ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))

# storing all the agent's actions into an excel sheet
ws.append(("Decision Variables",))
for i in act_store:
  ws.append(tuple(i))

ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))

In [ ]:
# saving of the excel workbook at the end of the last experient of an instance, the workbook is saved with the abbreviaed notation of the problem instance as the name
save_temp = sum(cap_dist) / max(cap_consum_mat)
wb.save(f'/content/drive/MyDrive/Dynamic_Programming_({no_aa},_{list(save_temp)},_{number_of_allocation_time_periods})_tensorboard_convergence_curves.xlsx')      # this statement wil only be exceuted at the end of the last experieent

# **Training of Value Iteration agent (by not exploiting the fact that our MDP is having finite horizon and therefore starting at an arbitrarily initialized Q-table while iteratively updating and improving it)**

In [3]:
# creating te excel sheet for insertion of data using "openpyxl"

wb = openpyxl.Workbook()     # creating a new workbook
ws = wb.active   # activating the workbook for reading and writing of data into and from the workbook respectively
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))

In [ ]:
is_loc_dec = False         # we are not using location allocation decisions in this research

gamma = 0.99999           # discount factor

no_dcl_loc_aval = 1        # number of local response centers (we are considering just one)
cap_consum_mat = np.array([1]).reshape(no_materials, no_dcl_loc_aval)                   # capacity consumption by a particular relief item in a particular local response center
Len = 4       # length (in terms of hours) of one time period
planning_horizon = Len * number_of_allocation_time_periods

episode_counter = 0

# enumeration of resource allocation decision from relief material allocation from local response centers to affected areas
counter = 0                                                                  # Its value will increase until all the combinations of the affected areas and distrubuiton centers are exhausted.

res_alocat_dc = []
m = 0
while m < no_aa:                                             # These nested while loops have the task to generate the numpy array list of all possible allocation aciton, which the agent can do. In it, there may be some actions which are infeasible.
  i = 0
  while i < no_materials:
    j = 0
    while j < no_dcl_loc_aval:
      k = 0
      res_alocat_dc.append([])
      while k <= np.floor(cap_dist[j] / cap_consum_mat[i, j]):
        res_alocat_dc[counter].append(k)
        k = k + 1
      counter = counter + 1
      j = j + 1
    i = i + 1
  m = m + 1
res_alocat_dc = np.array(res_alocat_dc)
alot_dcl_actions = np.array(np.meshgrid(*res_alocat_dc)).T.reshape(-1, no_dcl_loc_aval * no_aa * no_materials)                           # This statement creates the real numpy array list of all the possible resource allocation decisions. "alot_dcl_actions" contains all the possible actions related to resource allocation. The arrangement in "alot_dcl_actions" is no_materials by no_dcl_loc_aval.

# determination of action validaty (are all constraints satisfied or not?) and the elimination of invalid actions
del_el = []                  # array elements to be deleated
# if the allocation capacity of any distribution center location is exceeded by the total allocation made from that distribution ceneter location then the action is eliminated in the following code.
for j in range(0, alot_dcl_actions.shape[0]):          # Action shaping implementation. (eliminating some obvious infeasible actions) Ref: A. Kanervisto, C. Scheller and V. Hautamäki, "Action Space Shaping in Deep Reinforcement Learning," 2020 IEEE Conference on Games (CoG), 2020, pp. 479-486, doi: 10.1109/CoG47356.2020.9231687.
  compare_for_action_shaping = np.sum(np.multiply(np.reshape(np.sum(np.reshape(alot_dcl_actions[j], (no_aa, no_materials * no_dcl_loc_aval)), axis = 0), (no_materials, no_dcl_loc_aval)), cap_consum_mat), axis = 0)               # As capacity constraint is the hard constraint so we use action shaping to eliminate the actions which violate this constraint to ensure that the capacity constraints must be satisfied, otherwise it makes no sense that the allocation exceeds the capacity of distribution centers.
  for k in range(0, compare_for_action_shaping.shape[0]):
    if compare_for_action_shaping[k] > cap_dist[k]:
      del_el.append(j)
      break

alot_dcl_actions = np.delete(alot_dcl_actions, del_el, axis = 0)

del counter
del m
del i
del j
del k
del res_alocat_dc                                                          # Deleting the variables array as it is no longer required, to free up memory. (Make the code memory efficient)

alot_loc_dcl_actions_full_array = alot_dcl_actions   # if location allocation and relocation decisions are not included, then all the possible actions are only valid resource allocation actions

# enumeration of environment's state

# upper bound on the number of possible state
upper_bound_state_no = np.prod(number_of_allocation_time_periods * np.zeros_like(demand_aa_mat[0]) + 1 + np.abs(number_of_allocation_time_periods * np.zeros_like(demand_aa_mat[0]) - np.floor((np.max(cap_dist) / np.min(cap_consum_mat))) * number_of_allocation_time_periods)) * number_of_allocation_time_periods
state_storg_arr = np.zeros((int(upper_bound_state_no) , no_aa * no_materials + 1))

state_lst_storg = [[] for i in range(0, number_of_allocation_time_periods + 1)]     # this list of list will store only in the indices of the state arrays in "state_storg_arr" with respect to each time period
init_state = np.array([0 for i in range(0, no_aa * no_materials)])

counter_1 = 0       # this counter tracks the index in "state_storg_arr" where the states from the previous time period (which will be used along with actions to enumerate states for the current time period in our finite horizon MDP) starts
counter_2 = 0       # this counter tracks the index in "state_storg_arr"  where state from the previous the previous time period ends
counter_3 = 1       # this counter starts at the index where the next state (of the current time period) is to entered in "state_storg_arr"
for i in range(0, number_of_allocation_time_periods):           # loop over all the time periods
  for j in range(counter_1, counter_2 + 1):      # loop over all the states of the previous time period (i.e., between the index "counter_1" and "counter_2")
    for k in range(0, alot_loc_dcl_actions_full_array.shape[0]):       # loop over all the actions which the agent can take
      state_temp_temp = np.concatenate((state_storg_arr[j, 0 : no_aa * no_materials] - np.sum(np.reshape(alot_loc_dcl_actions_full_array[k], (no_aa * no_materials, no_dcl_loc_aval)), axis = 1) + demand_aa_mat[i], np.array([i + 1])), axis = 0)
      if np.sum(np.prod(state_storg_arr[counter_2 : counter_3,:] == state_temp_temp, axis = 1)) == 0:
        state_storg_arr[counter_3] = state_temp_temp
        state_lst_storg[i].append(counter_3)

        counter_3 = counter_3 + 1
        # state_storg_arr[: counter_3 - 1, :] = np.unique(state_storg_arr[: counter_3 - 1, :], axis = 0)
        # print(counter_3, "counter_2counter_2counter_2")
  counter_1 = counter_2 + 1
  counter_2 = counter_3 - 1

counter_1 = counter_1
state_storg_arr = state_storg_arr[: counter_3]      # removing all the rows in state storage array which are not used after all the possible states have been enumerated
# state_storg_arr = np.unique(state_storg_arr, axis = 0)    # removing all the rows which are not unique (untile they are unique)

state_lst_storg.insert(0, [0])

Ns = state_storg_arr.shape[0]
Na = alot_loc_dcl_actions_full_array.shape[0]

planning_horizon = Len * number_of_allocation_time_periods

In [ ]:
# a separate reward function for the environment
def reward_func(s, a, s_):
  # accessibility based delivery cost
  logistics_cost = np.dot(cost_path[int(state_storg_arr[s, -1]) - 1], alot_loc_dcl_actions_full_array[a])

  # deprivation cost
  deprivation_cost = 0
  counter = 0
  for st in state_storg_arr[s_, : no_aa * no_materials]:           # This for loop loops through all the affected areas and calculate the total deprivation cost.
    if st >= 0:                                                                   # It the state value of the affected area is greater then or equal to "0" then the formula below is used.
      deprivation_cost = deprivation_cost + exp(aaa[counter]) * (exp(bbb[counter] * Len) - 1) * (exp(bbb[counter] * Len) ** st)                        # This formula is used to calculate the deprivation cost for a particular Affected Area, given a particular state value of that Affected Area.
    if state_storg_arr[s, -1] == 0:                                                              # If it is the first time step of an episode then there is an additional step is calculating the deprivation cost for the time period.
      deprivation_cost = deprivation_cost + np.multiply(np.exp(aaa[counter]), (np.exp(bbb[counter] * Len) - 1))               # This is the additional step is calculating he deprivation cost for the first time period of the episode.
    counter = counter + 1
  return -(deprivation_cost_weight * deprivation_cost + accessibality_based_delivery_cost_weight * logistics_cost)

In [ ]:
# this will be used in value iteration algorithm
def bellman_operator(Q, greedy_policy):
  for s in range(0, state_storg_arr.shape[0]):              # loop over all the environment's states
    collection_next_state = np.array(state_lst_storg[int(state_storg_arr[s, -1] + 1)])
    if state_storg_arr[s, -1] != number_of_allocation_time_periods:
      for a in range(0, alot_loc_dcl_actions_full_array.shape[0]):              # loop over all the environment's actions
        value = 0
        for s_ in collection_next_state:
          if state_storg_arr[s, -1] != number_of_allocation_time_periods - 1:
            value = value + np.all(state_storg_arr[s_, 0 : no_aa * no_materials] == (state_storg_arr[s, 0 : no_aa * no_materials] - alot_loc_dcl_actions_full_array[a] + demand_aa_mat[int(state_storg_arr[s, -1])])) * (reward_func(s, a, s_) + gamma * np.max(Q[s_, :]))     # one step bootstrapping to approximate reward-to-go (in case of maximization) (cost-to-go in case of minimization, as is the case in our  code) function
          else:           # if the time period is the second last one in our MDP then no bootstraping is done as last states have zero reward and reward-to-go function
            value = value + np.all(state_storg_arr[s_, 0 : no_aa * no_materials] == (state_storg_arr[s, 0 : no_aa * no_materials] - alot_loc_dcl_actions_full_array[a] + demand_aa_mat[int(state_storg_arr[s, -1])])) * reward_func(s, a, s_)     # one step bootstrapping to approximate reward-to-go (in case of maximization) (cost-to-go in case of minimization, as is the case in our  code) function
          Q[s,a]  = value
    else:
      break

  greedy_policy = np.argmax(Q, axis = 1)           # output action (which can be taken by the agent) based on greedy policy

  return Q, greedy_policy          # updated q-table and greedy policy action

In [ ]:
def value_iteration(Q, greedy_policy):
  global term_n_steps
  counter = 0
  while True:
    Q, greedy_policy = bellman_operator(Q, greedy_policy)

    counter = counter + 1

    obj_val_val_iter = 0
    s = env.reset()     # initial state
    for j in range(0, number_of_allocation_time_periods):     # loop over all the time peirods inside an episode
      a = greedy_policy[s]
      s, reward, done, info = env.step(a)

      obj_val_val_iter = obj_val_val_iter + (gamma ** i) * reward

    # for te evaluatoin of the agent after predetermined number of steps
    ws.append((time.time(), counter, obj_val_val_iter))

    if counter > initial_steps_episod_stall:
      np_arr_eval[int(counter)] = obj_val_val_iter

      if (counter >= n_episod_stall):
        y = np.sort(np.abs(np_arr_eval[int(counter - n_episod_stall) : int(counter)] - np_arr_eval[int(counter)]))[0 : n_episod_stall - 100]
        if ((time.time() - st_time) >= 18000) or (np.max(y) <= delta_episod_stall):
          term_n_steps = counter
          return Q, greedy_policy

In [ ]:
st_time = time.time()

env = Envir(accessibality_based_delivery_cost_weight = accessibality_based_delivery_cost_weight, deprivation_cost_weight = deprivation_cost_weight, gamma = gamma,
            reallocate_t = 5, is_eval = False, is_loc_dec = False, is_use_dyn_prog = False)

np_arr_eval = np.zeros((int(10 ** 5),))
initial_steps_episod_stall = 1
delta_episod_stall = 50
n_episod_stall = 200
counter = 0

Q = np.zeros((Ns, Na))    # we are using "counter_1" instead of "env.Ns" because we cannot take any actions at the terminal states and thus their cost to go value must be zero (as there is no state after them)
greedy_policy = np.zeros(Ns)

ws.append(("Data for convergence curves: ",))
ws.append(("Wall Time", "Iteration", "Value"))   # for the creation of header in te excel sheet

Q, greedy_policy_val_iter = value_iteration(Q, greedy_policy)

exce_time = time.time() - st_time
print(exce_time)

In [ ]:
# number of time steps for which te algorithm is trained
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))
ws.append(("Training time: ", exce_time))
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))
ws.append(("Number of iterations for training: ", term_n_steps))
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))
print(term_n_steps)

## **Evaluation of Value Iteration Solution Policy**

In [ ]:
n_eval_ep = 1

obj_val_val_iter = 0

# for storing action during evaluation simulation
act_store = np.zeros((env.number_of_allocation_time_periods, env.no_aa * env.no_materials * env.no_dcl_loc_aval))

# for storing states during evaluation simulation
state_store = np.zeros((env.number_of_allocation_time_periods + 1, env.no_aa * env.no_materials))

ws.append(("Stagewise objective function value or reward:",))

for i in range(0, n_eval_ep):     # loop over the number of episodes
  s = env.reset()     # initial state
  state_store[0, :] = env.state_storg_arr[s, :env.no_aa * env.no_materials]
  for j in range(0, env.number_of_allocation_time_periods):     # loop over all the time peirods inside an episode
    a = greedy_policy_val_iter[s]
    act_store[j, :] = env.alot_loc_dcl_actions_full_array[a]
    s, reward, done, info = env.step(a)

    # storing all the stage wise objective values into an excel sheet
    ws.append((f"Stagewise objective function value at stage {j + 1}: ", reward))

    obj_val_val_iter = obj_val_val_iter + (gamma ** i) * reward
    state_store[j + 1, :] = env.state_storg_arr[s, :env.no_aa * env.no_materials]

In [ ]:
act_store

In [ ]:
# storing all the objective function valeu (sum of all the rewards) into an excel sheet
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))
ws.append(("Objective function value: ", obj_val_val_iter))
obj_val_val_iter
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))

In [ ]:
# storing all the state variables value to an excel sheet
ws.append(("State Variables",))
for i in state_store:
  ws.append(tuple(i))

ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))

# storing all the agent's actions into an excel sheet
ws.append(("Decision Variables",))
for i in act_store:
  ws.append(tuple(i))

ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))

In [ ]:
obj_val_val_iter

In [ ]:
# saving of the excel workbook at the end of the last experient of an instance, the workbook is saved with the abbreviaed notation of the problem instance as the name
save_temp = sum(cap_dist) / max(cap_consum_mat)
wb.save(f'/content/drive/MyDrive/Value_Iteration_({no_aa},_{list(save_temp)},_{number_of_allocation_time_periods})_tensorboard_convergence_curves.xlsx')      # this statement wil only be exceuted at the end of the last experieent

# **Training of SARSA agent**

In [11]:
# creating te excel sheet for insertion of data using "openpyxl"

wb = openpyxl.Workbook()     # creating a new workbook
ws = wb.active   # activating the workbook for reading and writing of data into and from the workbook respectively
ws.append(("####", "####", "####", "####", "####", "####", "####", "####", "####", f"Random seed: {seed}", "####", "####", "####", "####", "####", "####", "####", "####", "####"))
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))

In [12]:
from scipy.special import softmax

class Sarsa:
    """
    Implements SARSA algorithm.

    If learning_rate is None; alpha(x,a) = 1/max(1, N(s,a))**alpha
    """
    def __init__(self, env, gamma, alpha=0.6, learning_rate=None, min_learning_rate=0.01, tau=1.0, tau_decay=0.9995,
                 tau_min=0.25, seed=42):
        self.env = env
        self.gamma = gamma
        self.alpha = alpha
        self.learning_rate = learning_rate
        self.min_learning_rate = min_learning_rate
        self.tau = tau
        self.tau_decay = tau_decay
        self.tau_min = tau_min
        self.Q = np.zeros((env.Ns, env.Na))
        self.Nsa = np.zeros((env.Ns, env.Na))
        self.state = env.reset()
        self.RS = np.random.RandomState(seed)

    def get_delta(self, r, x, a, y, next_a, done):
        """
        :param r: reward
        :param x: current state
        :param a: current action
        :param y: next state
        :return:
        """
        q_y_a = self.Q[y, next_a]
        q_x_a = self.Q[x, a]

        return r + self.gamma*q_y_a - q_x_a

    def get_learning_rate(self, s, a):
        if self.learning_rate is None:
            return max(1.0/max(1.0, self.Nsa[s, a])**self.alpha, self.min_learning_rate)
        else:
            return max(self.learning_rate, self.min_learning_rate)

    def get_action(self, state):
        q = self.Q[state, :]
        prob = softmax(q/self.tau)
        a = np.random.choice(np.arange(0, self.env.alot_loc_dcl_actions_full_array.shape[0]))
        return a

    def step(self):
        # Current state
        x = self.env.state_indx_curr

        # Choose action
        a = self.get_action(x)

        # Learning rate
        alpha = self.get_learning_rate(x, a)

        # Take step
        observation, reward, done, info = self.env.step(a)
        y = observation
        r = reward
        next_a = self.get_action(y)
        delta = self.get_delta(r, x, a, y, next_a, done)

        # Update
        self.Q[x, a] = self.Q[x, a] + alpha*delta

        self.Nsa[x, a] += 1

        if done:
          # print(x, observation, reward)
          self.tau = max(self.tau*self.tau_decay, self.tau_min)
          self.env.reset()

In [13]:
# function for the evaluation of the policy

def evaluate_policy(gamma = 0.99999):
  global envv
  reward_sum = 0
  for i in range(0, number_of_allocation_time_periods):
    if i == 0:
      state = envv.reset()
    a = sarsa.Q[state, :].argmax()
    state, reward, done, info = envv.step(a)
    reward_sum = reward_sum + (gamma ** i) * reward
  return reward_sum

In [ ]:
# ---------------------------
# Convergence of SARSA
# ---------------------------

gamma = 0.99999
np_arr_eval = np.zeros((int(10 ** 5),))
save_freq = number_of_allocation_time_periods * 100
initial_steps_episod_stall = 10 ** 6
delta_episod_stall = 50
n_episod_stall = 200
count_eval = 0
term_n_steps = 0
st_time = time.time()

# Iterate
n_calls = 0
count_eval = 0

# crating environment object
env = Envir(accessibality_based_delivery_cost_weight = 1 / 3, deprivation_cost_weight = 1 / 3, gamma = gamma,
            reallocate_t = 5, is_eval = False, is_loc_dec = False, is_use_dyn_prog = False)

# crating environment object
envv = Envir(accessibality_based_delivery_cost_weight = 1 / 3, deprivation_cost_weight = 1 / 3, gamma = gamma,     # for evaluation only
            reallocate_t = 5, is_eval = False, is_loc_dec = False, is_use_dyn_prog = False)

# Get optimal value function and its greedy policy
Q0 = np.zeros((env.counter_1, env.Na))

# Create sarsa object
sarsa = Sarsa(env, gamma=env.gamma)

ws.append(("Data for convergence curves: ",))
ws.append(("Wall Time", "Step", "Value"))   # for the creation of header in te excel sheet

# Q_est = np.zeros((n_steps, env.counter_1, env.Na))
while True:

    if (n_calls > initial_steps_episod_stall) and (n_calls % save_freq == 0):
        np_arr_eval[int(count_eval)] = evaluate_policy()

        if (count_eval >= n_episod_stall):
          y = np.sort(np.abs(np_arr_eval[int(count_eval - n_episod_stall) : int(count_eval)] - np_arr_eval[int(count_eval)]))[0 : n_episod_stall - 100]
          if ((time.time() - st_time) >= 18000) or (np.max(y) <= delta_episod_stall):
            # print(((np.max(np.abs(np_arr_eval[int(count_eval - n_episod_stall) : int(count_eval)] - int(np_arr_eval[int(self.count_eval)]))) / int(np_arr_eval[int(self.count_eval)])) * 100), "qqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq")
            # print(int(np_arr_eval[int(count_eval)]), "dddddddddddddddddddddddddddddddddddddddddd")
            # print(np.max(np.abs(np_arr_eval[int(count_eval - n_episod_stall) : int(count_eval)] - int(np_arr_eval[int(count_eval)]))), "uuuuuuuuuuuuuuuuuuuuuu")
            term_n_steps = n_calls
            exce_time = time.time() - st_time
            print(exce_time)
            break
        count_eval = count_eval + 1

    sarsa.step()

    # Store estimate of Q*
    # Q_est[tt, :, :] = sarsa.Q
    n_calls +=1

    # for te evaluatoin of the agent after predetermined number of steps
    if n_calls % (1000 * number_of_allocation_time_periods) == 0:
      ws.append((time.time(), n_calls, evaluate_policy()))

# Compute greedy policy (with estimated Q)
greedy_policy_sarsa = np.argmax(sarsa.Q, axis=1)

In [ ]:
# number of time steps for which te algorithm is trained
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))
ws.append(("Training time: ", exce_time))
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))
ws.append(("Number of steps for training: ", n_calls))
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))
print(n_calls)

## **Evaluation of SARSA agnet**

In [ ]:
n_eval_ep = 1

obj_val_sarsa = 0

# for storing action during evaluation simulation
if env.is_loc_dec:
  act_store = np.zeros((env.number_of_allocation_time_periods, env.no_aa * env.no_materials * env.no_dcl_loc_aval + env.no_dcl_loc_aval))
else:
  act_store = np.zeros((env.number_of_allocation_time_periods, env.no_aa * env.no_materials * env.no_dcl_loc_aval))

# for storing states during evaluation simulation
if env.is_loc_dec:
  state_store = np.zeros((env.number_of_allocation_time_periods + 1, env.no_aa * env.no_materials + env.no_dcl_loc_aval))
else:
  state_store = np.zeros((env.number_of_allocation_time_periods + 1, env.no_aa * env.no_materials))

ws.append(("Stagewise objective function value or reward:",))

for i in range(0, n_eval_ep):     # loop over the number of episodes
  s = env.reset()     # initial state
  if env.is_loc_dec:
    state_store[0, :] = env.state_storg_arr[s, :env.no_aa * env.no_materials + env.no_dcl_loc_aval]
  else:
    state_store[0, :] = env.state_storg_arr[s, :env.no_aa * env.no_materials]
  for j in range(0, env.number_of_allocation_time_periods):     # loop over all the time peirods inside an episode
    a = greedy_policy_sarsa[s]
    act_store[j, :] = env.alot_loc_dcl_actions_full_array[a]
    s, reward, done, info = env.step(a)

    # storing all the stage wise objective values into an excel sheet
    ws.append((f"Stagewise objective function value at stage {j + 1}: ", reward))

    obj_val_sarsa = obj_val_sarsa + (gamma ** i) * reward
    if env.is_loc_dec:
      state_store[j + 1, :] = env.state_storg_arr[s, :env.no_aa * env.no_materials + env.no_dcl_loc_aval]
    else:
      state_store[j + 1, :] = env.state_storg_arr[s, :env.no_aa * env.no_materials]

In [ ]:
act_store

In [ ]:
# storing all the objective function valeu (sum of all the rewards) into an excel sheet
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))
ws.append(("Objective function value: ", obj_val_sarsa))
obj_val_sarsa
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))

In [ ]:
# storing all the state variables value to an excel sheet
ws.append(("State Variables",))
for i in state_store:
  ws.append(tuple(i))

ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))

# storing all the agent's actions into an excel sheet
ws.append(("Decision Variables",))
for i in act_store:
  ws.append(tuple(i))

ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))

In [ ]:
# saving of the excel workbook at the end of the last experient of an instance, the workbook is saved with the abbreviaed notation of the problem instance as the name
save_temp = sum(cap_dist) / max(cap_consum_mat)
wb.save(f'/content/drive/MyDrive/SARSA_agent_({no_aa},_{list(save_temp)},_{number_of_allocation_time_periods})_{seed}_tensorboard_convergence_curves.xlsx')      # this statement wil only be exceuted at the end of the last experieent

# **Training of Q-Learning agent**

In [15]:
# creating te excel sheet for insertion of data using "openpyxl"

wb = openpyxl.Workbook()     # creating a new workbook
ws = wb.active   # activating the workbook for reading and writing of data into and from the workbook respectively
ws.append(("####", "####", "####", "####", "####", "####", "####", "####", "####", f"Random seed: {seed}", "####", "####", "####", "####", "####", "####", "####", "####", "####"))
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))

In [16]:
#-------------------------------
# Q-Learning implementation
# ------------------------------

class QLearning:
    """
    Implements Q-learning algorithm with epsilon-greedy exploration

    If learning_rate is None; alpha(x,a) = 1/max(1, N(s,a))**alpha
    """
    def __init__(self, env, gamma, alpha=0.6, learning_rate=None, min_learning_rate=0.01, epsilon=1.0, epsilon_decay=0.9995,
                 epsilon_min=0.25, seed=42):
        self.env = env
        self.gamma = gamma
        self.alpha = alpha
        self.learning_rate = learning_rate
        self.min_learning_rate = min_learning_rate
        self.epsilon = epsilon
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.Q = np.zeros((env.Ns, env.Na))
        self.Nsa = np.zeros((env.Ns, env.Na))
        self.state = env.reset()
        self.RS = np.random.RandomState(seed)

    def get_delta(self, r, x, a, y, done):
        """
        :param r: reward
        :param x: current state
        :param a: current action
        :param y: next state
        :param done:
        :return:
        """
        max_q_y_a = self.Q[y, :].max()
        q_x_a = self.Q[x, a]

        return r + self.gamma*max_q_y_a - q_x_a

    def get_learning_rate(self, s, a):
        if self.learning_rate is None:
            return max(1.0/max(1.0, self.Nsa[s, a])**self.alpha, self.min_learning_rate)
        else:
            return max(self.learning_rate, self.min_learning_rate)

    def get_action(self, state):
        if self.RS.uniform(0, 1) < self.epsilon:
            # explore
            return np.random.choice(np.arange(0, self.env.alot_loc_dcl_actions_full_array.shape[0]))
        else:
            # exploit
            a = self.Q[state, :].argmax()
            return a

    def step(self):
        # Current state
        x = self.env.state_indx_curr

        # Choose action
        a = self.get_action(x)

        # Learning rate
        alpha = self.get_learning_rate(x, a)

        # Take step
        observation, reward, done, info = self.env.step(a)
        y = observation
        r = reward
        delta = self.get_delta(r, x, a, y, done)

        # Update
        self.Q[x, a] = self.Q[x, a] + alpha*delta

        self.Nsa[x, a] += 1

        if done:
            # print(x, observation, reward)
            self.epsilon = max(self.epsilon*self.epsilon_decay, self.epsilon_min)
            self.env.reset()

In [17]:
# function for the evaluation of the policy

def evaluate_policy(gamma = 0.99999):
  global envv
  reward_sum = 0
  for i in range(0, number_of_allocation_time_periods):
    if i == 0:
      state = envv.reset()
    a = qlearning.Q[state, :].argmax()
    state, reward, done, info = envv.step(a)
    reward_sum = reward_sum + (gamma ** i) * reward
  return reward_sum

In [ ]:
# ---------------------------
# Convergence of Q-Learning
# ---------------------------

gamma = 0.99999
np_arr_eval = np.zeros((int(10 ** 5),))
save_freq = number_of_allocation_time_periods * 100
initial_steps_episod_stall = 10 ** 6
delta_episod_stall = 50
n_episod_stall = 200
count_eval = 0
term_n_steps = 0
st_time = time.time()

# Iterate
n_calls = 0
count_eval = 0

# crating environment object
env = Envir(accessibality_based_delivery_cost_weight = 1 / 3, deprivation_cost_weight = 1 / 3, gamma = gamma,
            reallocate_t = 5, is_eval = False, is_loc_dec = False, is_use_dyn_prog = False)

# crating environment object
envv = Envir(accessibality_based_delivery_cost_weight = 1 / 3, deprivation_cost_weight = 1 / 3, gamma = gamma,     # for evaluation only
            reallocate_t = 5, is_eval = False, is_loc_dec = False, is_use_dyn_prog = False)

# Get optimal value function and its greedy policy
Q0 = np.zeros((env.counter_1, env.Na))

# Create qlearning object
qlearning = QLearning(env, gamma=env.gamma)

ws.append(("Data for convergence curves: ",))
ws.append(("Wall Time", "Step", "Value"))   # for the creation of header in te excel sheet

# Q_est = np.zeros((n_steps, env.counter_1, env.Na))
while True:

    if (n_calls > initial_steps_episod_stall) and (n_calls % save_freq == 0):
        np_arr_eval[int(count_eval)] = evaluate_policy()

        if (count_eval >= n_episod_stall):
          y = np.sort(np.abs(np_arr_eval[int(count_eval - n_episod_stall) : int(count_eval)] - np_arr_eval[int(count_eval)]))[0 : n_episod_stall - 100]
          if ((time.time() - st_time) >= 18000) or (np.max(y) <= delta_episod_stall):
            # print(((np.max(np.abs(np_arr_eval[int(count_eval - n_episod_stall) : int(count_eval)] - int(np_arr_eval[int(self.count_eval)]))) / int(np_arr_eval[int(self.count_eval)])) * 100), "qqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq")
            # print(int(np_arr_eval[int(count_eval)]), "dddddddddddddddddddddddddddddddddddddddddd")
            # print(np.max(np.abs(np_arr_eval[int(count_eval - n_episod_stall) : int(count_eval)] - int(np_arr_eval[int(count_eval)]))), "uuuuuuuuuuuuuuuuuuuuuu")
            term_n_steps = n_calls
            exce_time = time.time() - st_time
            print(exce_time)
            break
        count_eval = count_eval + 1

    qlearning.step()

    # Store estimate of Q*
    # Q_est[tt, :, :] = qlearning.Q
    n_calls +=1

    # for te evaluatoin of the agent after predetermined number of steps
    if n_calls % (1000 * number_of_allocation_time_periods) == 0:
      ws.append((time.time(), n_calls, evaluate_policy()))

# Compute greedy policy (with estimated Q)
greedy_policy_q_learning = np.argmax(qlearning.Q, axis=1)

In [ ]:
# number of time steps for which te algorithm is trained
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))
ws.append(("Training time: ", exce_time))
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))
ws.append(("Number of steps for training: ", n_calls))
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))
print(n_calls)

## **Evaluation of Q-Learning agent**

In [ ]:
n_eval_ep = 1

obj_val_q_learning = 0

# for storing action during evaluation simulation
if env.is_loc_dec:
  act_store = np.zeros((env.number_of_allocation_time_periods, env.no_aa * env.no_materials * env.no_dcl_loc_aval + env.no_dcl_loc_aval))
else:
  act_store = np.zeros((env.number_of_allocation_time_periods, env.no_aa * env.no_materials * env.no_dcl_loc_aval))

# for storing states during evaluation simulation
if env.is_loc_dec:
  state_store = np.zeros((env.number_of_allocation_time_periods + 1, env.no_aa * env.no_materials + env.no_dcl_loc_aval))
else:
  state_store = np.zeros((env.number_of_allocation_time_periods + 1, env.no_aa * env.no_materials))

ws.append(("Stagewise objective function value or reward:",))

for i in range(0, n_eval_ep):     # loop over the number of episodes
  s = env.reset()     # initial state
  if env.is_loc_dec:
    state_store[0, :] = env.state_storg_arr[s, :env.no_aa * env.no_materials + env.no_dcl_loc_aval]
  else:
    state_store[0, :] = env.state_storg_arr[s, :env.no_aa * env.no_materials]
  for j in range(0, env.number_of_allocation_time_periods):     # loop over all the time peirods inside an episode
    a = greedy_policy_q_learning[s]
    act_store[j, :] = env.alot_loc_dcl_actions_full_array[a]
    s, reward, done, info = env.step(a)

    # storing all the stage wise objective values into an excel sheet
    ws.append((f"Stagewise objective function value at stage {j + 1}: ", reward))

    obj_val_q_learning = obj_val_q_learning + (gamma ** i) * reward
    if env.is_loc_dec:
      state_store[j + 1, :] = env.state_storg_arr[s, :env.no_aa * env.no_materials + env.no_dcl_loc_aval]
    else:
      state_store[j + 1, :] = env.state_storg_arr[s, :env.no_aa * env.no_materials]

In [ ]:
act_store

In [ ]:
# storing all the objective function valeu (sum of all the rewards) into an excel sheet
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))
ws.append(("Objective function value: ", obj_val_q_learning))
obj_val_q_learning
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))

In [ ]:
# storing all the state variables value to an excel sheet
ws.append(("State Variables",))
for i in state_store:
  ws.append(tuple(i))

ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))

# storing all the agent's actions into an excel sheet
ws.append(("Decision Variables",))
for i in act_store:
  ws.append(tuple(i))

ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))
ws.append(("-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-", "-"))

In [ ]:
# saving of the excel workbook at the end of the last experient of an instance, the workbook is saved with the abbreviaed notation of the problem instance as the name
save_temp = sum(cap_dist) / max(cap_consum_mat)
wb.save(f'/content/drive/MyDrive/Q_learning_agent_({no_aa},_{list(save_temp)},_{number_of_allocation_time_periods})_{seed}_tensorboard_convergence_curves.xlsx')      # this statement wil only be exceuted at the end of the last experieent